# Using Tools from Composio

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1x2FYThMPtQXLzKZAhf_Ry-oeU6oPygdm?usp=sharing)

⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)

*Goal: Star a repository on GitHub with natural language & CAMEL Agent*

### Install Packages & Connect a Tool

Integrate Composio with CAMEL agents to let them seamlessly interact with external apps

Ensure you have the necessary packages installed and connect your GitHub account to allow your CAMEL-AI agents to utilize GitHub functionalities.

In [ ]:
%pip install "camel-ai[all]==0.1.6.5"
%pip install "composio-camel -U"

import composio


In [ ]:
# Login to Composio
!composio login

In [ ]:
# Connect your Github account (this is a shell command, so it should be run in your terminal or with '!' prefix in a Jupyter Notebook)
!composio add github

# Check all different apps which you can connect with
!composio apps


> Do you want to replace the existing connection? (y, n): y

> Adding integration: Github...

Please authenticate github in the browser and come back here. URL: https://github.com/login/oauth/authorize?client_id=37beeaa7020925b04e40&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=repo%2Cadmin%3Aorg%2Cgist%2Cuser&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiIwMWIyMTlmOC1kZTk5LTQ5ZWMtOTMwOS1mMzg3ZWFhMzRmNTgiLCJpbnRlZ3JhdGlvbklkIjoiMjdmNzM0YzAtZTIyMS00OTI1LTgxMjAtN2I1ZDNlYjAxOTA5IiwiYXBwTmFtZSI6ImdpdGh1YiIsImNsaWVudEluZm8iOnsiY2xpZW50SWQiOiJjZmE4NWM0ZC1jMWI0LTRjNmQtODE4Ni1kNmNiNmRmYmZlM2UifSwiaWF0IjoxNzIyMjE0NzUwfQ.w9QHilHJcHk8eIQa7vFCrGskC-VRFzgJIu1dQf4e7Jk&code_challenge=FnINCywVMtW7jH3DOGw6ddbmTf5vAJw5NvpojJkB2hs&code_challenge_method=S256
⚠ Waiting for github authentication...
✔ github added successfully!
Showing all apps
• multionai
• typeform
• dropbox
• slack
• yousearch
• hackernews
• scheduler
• apify
• googlecale

In [ ]:
# Update Composio apps
! composio apps update

⚠️ Apps does not require update
⚠️ Tags does not require update
⚠️ Actions does not require update
⚠️ Triggers does not require update


### Prepare your environment by initializing necessary imports from CAMEL & Composio.

In [ ]:
from typing import List

from colorama import Fore
from composio_camel import Action, ComposioToolSet

from camel.agents.chat_agent import FunctionCallingRecord
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.societies import RolePlaying
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated

In [ ]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Alternatively, if running on Colab, you could save your API keys and tokens as **Colab Secrets**, and use them across notebooks.

To do so, **comment out** the above **manual** API key prompt code block(s), and **uncomment** the following codeblock.

⚠️ Don't forget granting access to the API key you would be using to the current notebook.

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Let's run CAMEL agents with tools from Composio!



In [ ]:
# Set your task

task_prompt = (
    "I have created a new Github Repo,"
    "Please star my github repository: camel-ai/camel"
)

In [ ]:
# Set Toolset

composio_toolset = ComposioToolSet()

tools = composio_toolset.get_actions(
    actions=[Action.GITHUB_STAR_A_REPOSITORY_FOR_THE_AUTHENTICATED_USER]
)

In [ ]:
# Set models for user agent and assistant agent, give tool to the assistant

assistant_agent_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_3_5_TURBO,
    model_config_dict=ChatGPTConfig(tools=tools).as_dict(),
)

user_agent_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_3_5_TURBO,
    model_config_dict=ChatGPTConfig().as_dict(),
)

In [ ]:
# Set RolePlaying session

role_play_session = RolePlaying(
    assistant_role_name="Developer",
    user_role_name="CAMEL User",
    assistant_agent_kwargs=dict(
        model=assistant_agent_model,
        tools=tools,
    ),
    user_agent_kwargs=dict(
        model=user_agent_model,
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
)

In [ ]:
# Print the system message and task prompt

print(
    Fore.GREEN
    + f"AI Assistant sys message:\n{role_play_session.assistant_sys_msg}\n"
)
print(Fore.BLUE + f"AI User sys message:\n{role_play_session.user_sys_msg}\n")

print(Fore.YELLOW + f"Original task prompt:\n{task_prompt}\n")
print(
    Fore.CYAN
    + "Specified task prompt:"
    + f"\n{role_play_session.specified_task_prompt}\n"
)
print(Fore.RED + f"Final task prompt:\n{role_play_session.task_prompt}\n")

AI Assistant sys message:
BaseMessage(role_name='Developer', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': 'I have created a new Github Repo,Please star my github repository: camel-ai/camel', 'assistant_role': 'Developer', 'user_role': 'CAMEL User'}, content='===== RULES OF ASSISTANT =====\nNever forget you are a Developer and I am a CAMEL User. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: I have created a new Github Repo,Please star my github repository: camel-ai/camel. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately solves the requested instruction and explain your solutions.\nYou must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal re

In [ ]:
# Set terminate rule and print the chat message

n = 0
input_msg = role_play_session.init_chat()
while n < 50:
    n += 1
    assistant_response, user_response = role_play_session.step(input_msg)

    if assistant_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI Assistant terminated. Reason: "
                f"{assistant_response.info['termination_reasons']}."
            )
        )
        break
    if user_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI User terminated. "
                f"Reason: {user_response.info['termination_reasons']}."
            )
        )
        break

    # Print output from the user
    print_text_animated(
        Fore.BLUE + f"AI User:\n\n{user_response.msg.content}\n"
    )

    # Print output from the assistant, including any function
    # execution information
    print_text_animated(Fore.GREEN + "AI Assistant:")
    tool_calls: List[FunctionCallingRecord] = assistant_response.info[
        'tool_calls'
    ]
    for func_record in tool_calls:
        print_text_animated(f"{func_record}")
    print_text_animated(f"{assistant_response.msg.content}\n")

    if "CAMEL_TASK_DONE" in user_response.msg.content:
        break

    input_msg = assistant_response.msg

AI User:

Instruction: Visit the Github repository I created.
Input: None


AI Assistant:

Solution: To visit the Github repository you created, you can simply open a web browser and go to the following URL: https://github.com/camel-ai/camel

Next request.


AI User:

Instruction: Click on the "Star" button on the Github repository page.
Input: None


AI Assistant:

Function Execution: github_activity_star_repo_for_authenticated_user
	Args: {'owner': 'camel-ai', 'repo': 'camel'}
	Result: {'execution_details': {'executed': True}, 'response_data': ''}

Solution: I have successfully starred your Github repository "camel-ai/camel".

Next request.


AI User:

<CAMEL_TASK_DONE>


AI Assistant:

Great! The task is completed. If you have any more tasks or need further assistance in the future, feel free to ask. Have a wonderful day!


